In [19]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 数据转换：将图像数据转换为张量，并进行归一化
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 加载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# 定义预处理步骤
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     Flatten()])

In [21]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

# CIFAR-10图片尺寸为32x32x3
input_size = 32 * 32 * 3
num_classes = 10
model = LogisticRegressionModel(input_size, num_classes)


In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [23]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.view(inputs.size(0), -1)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:    # 每100个批次打印一次
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training')


Epoch [1/5], Step [100], Loss: 2.0180
Epoch [1/5], Step [200], Loss: 1.8527
Epoch [1/5], Step [300], Loss: 1.8432
Epoch [1/5], Step [400], Loss: 1.8096
Epoch [1/5], Step [500], Loss: 1.7945
Epoch [2/5], Step [100], Loss: 1.7908
Epoch [2/5], Step [200], Loss: 1.7700
Epoch [2/5], Step [300], Loss: 1.7698
Epoch [2/5], Step [400], Loss: 1.7551
Epoch [2/5], Step [500], Loss: 1.7537
Epoch [3/5], Step [100], Loss: 1.7605
Epoch [3/5], Step [200], Loss: 1.7322
Epoch [3/5], Step [300], Loss: 1.7268
Epoch [3/5], Step [400], Loss: 1.7347
Epoch [3/5], Step [500], Loss: 1.7441
Epoch [4/5], Step [100], Loss: 1.7258
Epoch [4/5], Step [200], Loss: 1.7326
Epoch [4/5], Step [300], Loss: 1.7283
Epoch [4/5], Step [400], Loss: 1.7112
Epoch [4/5], Step [500], Loss: 1.7125
Epoch [5/5], Step [100], Loss: 1.7191
Epoch [5/5], Step [200], Loss: 1.7060
Epoch [5/5], Step [300], Loss: 1.6977
Epoch [5/5], Step [400], Loss: 1.7062
Epoch [5/5], Step [500], Loss: 1.7182
Finished Training


In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.view(images.size(0), -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the model on the 10000 test images: 40.46%


Files already downloaded and verified
Files already downloaded and verified


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x100 and 3072x10)